# GOOGLE MAP API

# IMPORT LIBS

In [9]:
import pandas as pd
import json
import requests

# OPEN DATASET

In [10]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv', nrows=10000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance (miles)
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.639764
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,0.863167
2,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,1.737223
3,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.241710
4,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,2.353742
...,...,...,...,...,...,...,...,...,...
9995,2009-11-17 09:01:00.00000093,6.1,2009-11-17 09:01:00 UTC,-73.963182,40.763158,-73.977597,40.753898,1,0.990121
9996,2010-07-09 22:55:00.00000045,20.5,2010-07-09 22:55:00 UTC,-74.000030,40.730858,-73.989062,40.662120,2,4.777913
9997,2014-12-22 17:46:00.000000151,12.0,2014-12-22 17:46:00 UTC,-73.998345,40.745617,-73.981033,40.767580,1,1.766894
9998,2010-05-20 22:41:00.00000028,6.1,2010-05-20 22:41:00 UTC,-73.981652,40.756235,-73.967163,40.757127,2,0.762739


In [11]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
API_key = get_keys("../../ASSETS/GMAP/api_key.json")
google_key = list(API_key.values())[0]

In [12]:
lat_origin = df['pickup_latitude'].tolist()
long_origin = df['pickup_longitude'].tolist()
lat_destination = df['dropoff_latitude'].tolist()
long_destination = df['dropoff_longitude'].tolist()

distance = []
duration = []
for i in range(len(long_destination)):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={lat_origin[i]},{long_origin[i]}&destinations={lat_destination[i]}%2C{long_destination[i]}&key={google_key}"
    r=requests.get(url)
    data = r.json()
    try:
        distance.append(data['rows'][0]['elements'][0]['distance']['text'])
        duration.append(data['rows'][0]['elements'][0]['duration']['text'])
    except:
        pass

In [15]:
print(len(distance))

9999


In [16]:
distance

['0.9 mi',
 '0.9 mi',
 '2.4 mi',
 '1.2 mi',
 '2.8 mi',
 '1.2 mi',
 '3.8 mi',
 '1.5 mi',
 '2.5 mi',
 '1.5 mi',
 '0.6 mi',
 '1.7 mi',
 '1.4 mi',
 '2.2 mi',
 '1.2 mi',
 '1.0 mi',
 '0.5 mi',
 '1.7 mi',
 '1.9 mi',
 '0.8 mi',
 '0.9 mi',
 '1.0 mi',
 '1.6 mi',
 '0.7 mi',
 '3.1 mi',
 '0.7 mi',
 '0.7 mi',
 '5.8 mi',
 '1.2 mi',
 '6.1 mi',
 '0.5 mi',
 '0.8 mi',
 '1.9 mi',
 '2.9 mi',
 '1.4 mi',
 '3.2 mi',
 '3.8 mi',
 '3.5 mi',
 '0.9 mi',
 '2.0 mi',
 '0.6 mi',
 '0.4 mi',
 '0.6 mi',
 '1.6 mi',
 '2.0 mi',
 '0.8 mi',
 '1.6 mi',
 '0.3 mi',
 '4.3 mi',
 '1.4 mi',
 '5.2 mi',
 '1.1 mi',
 '0.4 mi',
 '0.8 mi',
 '0.9 mi',
 '1.0 mi',
 '1.9 mi',
 '1.1 mi',
 '1.0 mi',
 '2.9 mi',
 '3.2 mi',
 '0.7 mi',
 '0.7 mi',
 '0.5 mi',
 '0.9 mi',
 '1.3 mi',
 '4.9 mi',
 '2.9 mi',
 '1.3 mi',
 '1.5 mi',
 '1.6 mi',
 '1.2 mi',
 '1.0 mi',
 '1.2 mi',
 '4.8 mi',
 '1.7 mi',
 '1.4 mi',
 '3.5 mi',
 '1.6 mi',
 '1.5 mi',
 '0.7 mi',
 '2.8 mi',
 '1.7 mi',
 '2.0 mi',
 '1.1 mi',
 '1.7 mi',
 '2.8 mi',
 '1.2 mi',
 '1.3 mi',
 '4.0 mi',
 '1.1 mi',

In [ ]:
duration

In [17]:
last = len(df)
df = df.drop(df.index[last-1])

In [18]:
distance_cleaned = []
duration_cleaned = []
for i in range(len(distance)):
    distance_clean = distance[i].replace(' mi', '').replace(' ft', '')
    float_distance_clean = float(distance_clean)
    distance_cleaned.append(float_distance_clean)
    duration_clean = duration[i].replace(' mins', '').replace(' min', '')
    float_duration_clean = float(duration_clean)
    duration_cleaned.append(float_duration_clean)


df['Driving distance (miles)'] = distance_cleaned
df['Driving duration'] = duration_cleaned
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance (miles),Driving distance (miles),Driving duration
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.639764,0.9,5.0
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,0.863167,0.9,7.0
2,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,1.737223,2.4,18.0
3,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.241710,1.2,5.0
4,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,2.353742,2.8,17.0


In [20]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01a_geo_gmap.csv', index=False)